# STOCK PRICE PREDICTION MODEL
#### By Abhishek Aman

In [2]:
#importing the necessary libraries
import pandas as pd
import numpy as np
from  sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.preprocessing import StandardScaler

In [3]:
#loading the data
data_raw=pd.read_csv('stocks.csv',index_col='Date')
# glimpse of the dataset
data_raw.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1988-01-04,1952.589966,2030.010010,1950.760010,2015.250000,2015.250000,20880000
1988-01-05,2056.370117,2075.270020,2021.390015,2031.500000,2031.500000,27200000
1988-01-06,2036.469971,2058.189941,2012.770020,2037.800049,2037.800049,18800000
1988-01-07,2019.890015,2061.510010,2004.640015,2051.889893,2051.889893,21370000
1988-01-08,2046.579956,2058.689941,1898.040039,1911.310059,1911.310059,27440000


#### Creating different features to help in prediction model  

In [4]:
#creating different features from the given dataset
def generate_features(df):
    df_new=pd.DataFrame()
    df_new['open']=df['Open']
    #to get different stock parameters of previous day
    df_new['open_1']=df['Open'].shift(1) 
    df_new['close_1']=df['Close'].shift(1)
    df_new['high_1']=df['High'].shift(1)
    df_new['low_1']=df['Low'].shift(1)
    df_new['volume_1']=df['Volume'].shift(1)
    # average price of stock for 5 days,a month and whole year
    df_new['avg_price_5']=df['Close'].rolling(5).mean().shift(1)
    df_new['avg_price_30']=df['Close'].rolling(21).mean().shift(1)
    df_new['avg_price_365']=df['Close'].rolling(252).mean().shift(1)
    #finding the ratio of the above parameters
    df_new['ratio_avg_price_5_30']=df_new['avg_price_5']/df_new['avg_price_30']
    df_new['ratio_avg_price_5_365']=df_new['avg_price_5']/df_new['avg_price_365']
    df_new['ratio_avg_price_30_365']=df_new['avg_price_30']/df_new['avg_price_365']
    #average volume of stock traded for 5 days,a month and whole year
    df_new['avg_volume_5']=df['Volume'].rolling(5).mean().shift(1)
    df_new['avg_volume_30']=df['Volume'].rolling(21).mean().shift(1)
    df_new['avg_volume_365']=df['Volume'].rolling(252).mean().shift(1)
    #finding the ratio of the above parameters
    df_new['ratio_avg_volume_5_30']=df_new['avg_volume_5']/df_new['avg_volume_30']
    df_new['ratio_avg_volume_5_365']=df_new['avg_volume_5']/df_new['avg_volume_365']
    df_new['ratio_avg_volume_30_365']=df_new['avg_volume_30']/df_new['avg_volume_365']
    #standard deviation of prices of stock
    df_new['std_price_5']=df['Close'].rolling(5).std().shift(1)
    df_new['std_price_30']=df['Close'].rolling(21).std().shift(1)
    df_new['std_price_365']=df['Close'].rolling(252).std().shift(1)
    #finding the ratio of the above parameters
    df_new['ratio_std_price_5_30']=df_new['std_price_5']/df_new['std_price_30']
    df_new['ratio_std_price_5_365']=df_new['std_price_5']/df_new['std_price_365']
    df_new['ratio_std_price_30_365']=df_new['std_price_30']/df_new['std_price_365']
    #standard deviation of volume of stock traded
    df_new['std_volume_5']=df['Volume'].rolling(5).std().shift(1)
    df_new['std_volume_30']=df['Volume'].rolling(21).std().shift(1)
    df_new['std_volume_365']=df['Volume'].rolling(252).std().shift(1)
    #finding the ratio of the above parameters
    df_new['ratio_std_volume_5_30']=df_new['std_volume_5']/df_new['std_volume_30']
    df_new['ratio_std_volume_5_365']=df_new['std_volume_5']/df_new['std_volume_365']
    df_new['ratio_std_volume_30_365']=df_new['std_volume_30']/df_new['std_volume_365']
    # the Target
    df_new['close']=df['Close']
    df_new=df_new.dropna(axis=0)
    return df_new
    
    
     


In [5]:
#assigning the features to the dataset
data= generate_features(data_raw)
data.head()

,open,open_1,close_1,high_1,low_1,volume_1,avg_price_5,avg_price_30,avg_price_365,ratio_avg_price_5_30,...,ratio_std_price_5_30,ratio_std_price_5_365,ratio_std_price_30_365,std_volume_5,std_volume_30,std_volume_365,ratio_std_volume_5_30,ratio_std_volume_5_365,ratio_std_volume_30_365,close
Date,,,,,,,,,,,,,,,,,,,,,
1988-12-30,2183.389893,2169.110107,2182.679932,2193.040039,2165.179932,12220000.0,2168.215967,2146.005697,2061.050753,1.010350,...,0.372085,0.123335,0.331471,4.260343e+06,4.572098e+06,6.461774e+06,0.931814,0.659315,0.707561,2168.570068
1989-01-03,2163.209961,2183.389893,2168.570068,2193.750000,2162.500000,11140000.0,2169.857959,2148.579985,2061.659166,1.009903,...,0.332631,0.106706,0.320794,2.605936e+06,4.627472e+06,6.476618e+06,0.563145,0.402361,0.714489,2144.639893
1989-01-04,2153.750000,2163.209961,2144.639893,2168.389893,2127.139893,17310000.0,2164.999951,2150.616176,2062.108134,1.006688,...,0.677084,0.191809,0.283287,3.633685e+06,4.665697e+06,6.453401e+06,0.778809,0.563065,0.722983,2177.679932
1989-01-05,2184.290039,2153.750000,2177.679932,2183.389893,2146.610107,15710000.0,2167.999951,2154.682838,2062.663213,1.006181,...,0.914097,0.204474,0.223690,3.057422e+06,4.671999e+06,6.455623e+06,0.654414,0.473606,0.723710,2190.540039
1989-01-06,2195.889893,2184.290039,2190.540039,2205.179932,2173.040039,20310000.0,2172.821973,2157.862840,2063.213412,1.006932,...,1.089841,0.245020,0.224822,3.744766e+06,4.825499e+06,6.454034e+06,0.776037,0.580221,0.747672,2194.290039


#### Creating dataset for testing and training dataset

In [6]:
# from 1998-2015 is set as train data
start_train="1998-01-01"
end_train="2015-12-31"
# 2016 year data is used as test data
start_test="2016-01-01"
end_test="2016-12-31"

#creating the train dataset
data_train= data[start_train:end_train]
X_train= data_train.drop('close',axis=1).values
y_train= data_train['close'].values

#creating the test dataset
data_test= data[start_test:end_test]
X_test= data_test.drop('close',axis=1).values
y_test= data_test['close'].values

#### Use of Liner Regression to create the model

In [7]:
#using standardscaler technique to standardize all values
scaler=StandardScaler()
#Using the train data to fit it with scaler object
X_scaled_train= scaler.fit_transform(X_train) 
#Imputing the test data on the scaler object
X_scaled_test=scaler.transform(X_test)

param_grid={
            "alpha":[1e-5, 3e-5, 1e-4],
            "eta0":[0.01,0.03,0.1]
}
# Using Stochastic Gradient Descent to find the optimal value
from sklearn.linear_model import SGDRegressor
lr=SGDRegressor(max_iter=150, penalty="l2")
grid_search=GridSearchCV(lr, param_grid,cv=5,scoring="r2")
grid_search.fit(X_scaled_train,y_train)
print(grid_search.best_params_)

c:\Users\aman9\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


{'alpha': 0.0001, 'eta0': 0.03}


In [8]:
#using the the scaled data to get the prediction value
lr_best= grid_search.best_estimator_
predictions_lr=lr_best.predict(X_scaled_test)

In [9]:
# to find the value of different error
print('MSE:',(mean_squared_error(y_test,predictions_lr)))
print('MAE:',(mean_absolute_error(y_test,predictions_lr)))
# the value of 0.97 for r^2 gives us the idea how accurate is the model in predicitng stock value
print('R^2:',(r2_score(y_test,predictions_lr)))

MSE: 20791.48419448777
MAE: 110.42331570046242
R^2: 0.9765137185804557


#### Use of Random Forest to create the model

In [10]:
# providing necessary parameter for the RF model
param_grid={
            'max_depth':[5,10,15],
            'min_samples_split':[5,10],
            'max_features':['auto','sqrt'],
           'min_samples_leaf':[3,4]
          }
          

In [18]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor(n_estimators=100,n_jobs=-1)
grid_search=GridSearchCV(rf, param_grid, cv=5, scoring="r2", n_jobs=-1)
grid_search.fit(X_train,y_train)
rf_best= grid_search.best_estimator_
predictions_rf=rf_best.predict(X_test)


In [20]:
print('MSE:',(mean_squared_error(y_test,predictions_rf)))
print('MAE:',(mean_absolute_error(y_test,predictions_rf)))
# the value of r^2 of 0.7 is less than that of the LR model which means its relatively less accurate in predicitng price value
print('R^2:',(r2_score(y_test,predictions_rf)))

MSE: 262985.3767963617
MAE: 300.92987930650537
R^2: 0.7029289246074231
